<a href="https://colab.research.google.com/github/Bhavya-2k03/Deep-Learning/blob/main/v3(on_webcam).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import time
import os
from os.path import join,isfile
from os import listdir
from google.colab.patches import cv2_imshow

In [ ]:
!wget https://moderncomputervision.s3.eu-west-2.amazonaws.com/YOLO.zip

--2022-07-08 05:06:54--  https://moderncomputervision.s3.eu-west-2.amazonaws.com/YOLO.zip
Resolving moderncomputervision.s3.eu-west-2.amazonaws.com (moderncomputervision.s3.eu-west-2.amazonaws.com)... 52.95.150.42
Connecting to moderncomputervision.s3.eu-west-2.amazonaws.com (moderncomputervision.s3.eu-west-2.amazonaws.com)|52.95.150.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 231559154 (221M) [application/zip]
Saving to: ‘YOLO.zip’

YOLO.zip            100%[===================>] 220.83M  36.2MB/s    in 6.9s    

2022-07-08 05:07:02 (32.1 MB/s) - ‘YOLO.zip’ saved [231559154/231559154]



In [ ]:
! unzip YOLO.zip

Archive:  YOLO.zip
   creating: YOLO/
  inflating: YOLO/.DS_Store          
  inflating: __MACOSX/YOLO/._.DS_Store  
   creating: YOLO/images/
   creating: YOLO/yolo/
  inflating: YOLO/images/londonxmas2.jpeg  
  inflating: __MACOSX/YOLO/images/._londonxmas2.jpeg  
  inflating: YOLO/images/coffee_view.jpeg  
  inflating: __MACOSX/YOLO/images/._coffee_view.jpeg  
  inflating: YOLO/images/coffee.jpg  
  inflating: __MACOSX/YOLO/images/._coffee.jpg  
  inflating: YOLO/images/iceland_rum.jpeg  
  inflating: __MACOSX/YOLO/images/._iceland_rum.jpeg  
  inflating: YOLO/images/rot.jpg     
  inflating: __MACOSX/YOLO/images/._rot.jpg  
  inflating: YOLO/images/tommys_beers.jpeg  
  inflating: __MACOSX/YOLO/images/._tommys_beers.jpeg  
  inflating: YOLO/images/Volleyball.jpeg  
  inflating: __MACOSX/YOLO/images/._Volleyball.jpeg  
  inflating: YOLO/images/truck.jpg   
  inflating: __MACOSX/YOLO/images/._truck.jpg  
  inflating: YOLO/images/flowers.jpeg  
  inflating: __MACOSX/YOLO/images/._flowe

In [ ]:

labels_path="/content/YOLO/yolo/coco.names"
labels=open(labels_path).read().strip().split('\n')

colors=np.random.randint(0,255,size=(len(labels),3),dtype='uint8')
print(len(colors))

net=cv2.dnn.readNetFromDarknet("/content/YOLO/yolo/yolov3.cfg","/content/YOLO/yolo/yolov3.weights")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)


80


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:

image=cv2.imread("/content/photo.jpg")
(H,W)=image.shape[:2]
blob=cv2.dnn.blobFromImage(image,1/255,(416,416),swapRB=True,crop=False)
net.setInput(blob)
out_ln=net.getUnconnectedOutLayersNames()
layer_outputs=net.forward(out_ln)
boxe_s=[]
classids=[]
confidence_s=[]
for outputs in layer_outputs:
  for detection in outputs:
   scores=detection[5:]
   id =np.argmax(scores)
   confidence=max(scores)

   if confidence>0.75:
     box=detection[0:4]*np.array([W,H,W,H])
     (centre_x,centre_y,width,height)=box.astype("int")
     x=(centre_x-width/2)
     y=(centre_y-height/2)
     boxe_s.append([int(x),int(y),int(width),int(height)]) 
     classids.append(id)
     confidence_s.append(float(confidence))
idxs=cv2.dnn.NMSBoxes(boxe_s,confidence_s,0.5,0.3)

if len(idxs)>0 :
  for i in idxs.flatten():
   (x,y)=boxe_s[i][0],boxe_s[i][1]
   (width,height)=boxe_s[i][2],boxe_s[i][3]
   colo_r=[int(c) for c in colors[classids[i]]]
   cv2.rectangle(image,(x,y),(x+width,y+width),colo_r,3)
   text="{}: {:.4f}".format(labels[classids[i]],confidence_s[i])
   cv2.putText(image,text,(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,colo_r,2)  

cv2_imshow(image)
cv2.imwrite("Output.jpg", image)

from google.colab import files
files.download("Output.jpg")


